# 金融异常检测任务

## 1. 实验介绍

反欺诈是金融行业永恒的主题，在互联网金融信贷业务中，数字金融反欺诈技术已经得到广泛应用并取得良好效果，这其中包括了近几年迅速发展并在各个领域
得到越来越广泛应用的神经网络。本项目以互联网智能风控为背景，从用户相互关联和影响的视角，探索满足风控反欺诈领域需求的，可拓展、高效的神经
网络应用方案，从而帮助更好地识别欺诈用户。

本项目主要关于实现预测模型(**项目用图神经网络举例，具体实现可以使用其他模型**)，进行节点异常检测任务，并验证模型精度。而本项目基于的数据集[DGraph](https://dgraph.xinye.com/introduction)，[DGraph](https://dgraph.xinye.com/introduction)
是大规模动态图数据集的集合，由真实金融场景中随着时间演变事件和标签构成。

### 1.1 实验目的

- 了解如何使用Pytorch进行神经网络训练
- 了解如何使用Pytorch-geometric等图网络深度学习库进行简单图神经网络设计(推荐使用GAT, GraphSAGE模型)。
- 了解如何利用MO平台进行模型性能评估。

### 1.2 预备知识
- 具备一定的深度学习理论知识，如卷积神经网络、损失函数、优化器，训练策略等。
- 了解并熟悉Pytorch计算框架。
- 学习Pytorch-geometric，请前往：https://pytorch-geometric.readthedocs.io/en/latest/
    
### 1.3实验环境
- numpy = 1.26.4  
- pytorch = 2.3.1  
- torch_geometric = 2.5.3  
- torch_scatter = 2.1.2  
- torch_sparse = 0.6.18  

## 2. 实验内容

### 2.1 数据集信息
DGraph-Fin 是一个由数百万个节点和边组成的有向无边权的动态图。它代表了Finvolution Group用户之间的社交网络，其中一个节点对应一个Finvolution 用户，从一个用户到另一个用户的边表示**该用户将另一个用户视为紧急联系人**。
下面是`位于dataset/DGraphFin目录`的DGraphFin数据集的描述:
```
x:  20维节点特征向量
y:  节点对应标签，一共包含四类。其中类1代表欺诈用户而类0代表正常用户(实验中需要进行预测的两类标签)，类2和类3则是背景用户，即无需预测其标签。
edge_index:  图数据边集,每条边的形式(id_a,id_b)，其中ids是x中的索引
edge_type: 共11种类型的边
edge_timestamp: 脱敏后的时间戳
train_mask, valid_mask, test_mask: 训练集，验证集和测试集掩码
```
本预测任务为识别欺诈用户的节点预测任务,只需要将欺诈用户（Class 1）从正常用户（Class 0）中区分出来。需要注意的是，其中测试集中样本对应的label**均被标记为-100**。

### 2.2 导入相关包

导入相应模块，设置数据集路径、设备等。

In [1]:
from utils import DGraphFin
from utils.utils import prepare_folder
from utils.evaluator import Evaluator

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T

import numpy as np
from torch_geometric.data import Data
import os

#设置gpu设备
device = 0
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
print(f'Using {device}')

Using cuda:0


### 2.3 数据处理

在使用数据集训练网络前，首先需要对数据进行归一化等预处理，如下：

In [2]:
path='./datasets/632d74d4e2843a53167ee9a1-momodel/' #数据保存路径
save_dir='./results/' #模型保存路径
dataset_name='DGraph'
# dataset = DGraphFin(root=path, name=dataset_name, transform=T.Compose([T.ToUndirected(),T.ToSparseTensor()]))
# dataset = DGraphFin(root=path, name=dataset_name, transform= T.Compose([T.ToUndirected(),T.ToSparseTensor()]))
dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor())
nlabels = dataset.num_classes
if dataset_name in ['DGraph']:
    nlabels = 2    #本实验中仅需预测类0和类1

data = dataset[0]
data.adj_t = data.adj_t.to_symmetric() #将有向图转化为无向图


if dataset_name in ['DGraph']:
    x = data.x
    x = (x - x.mean(0)) / x.std(0)
    data.x = x
if data.y.dim() == 2:
    data.y = data.y.squeeze(1)

split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  #划分训练集，验证集

train_idx = split_idx['train']
result_dir = prepare_folder(dataset_name,'mlp')


这里我们可以查看数据各部分维度

In [3]:
print(data)
print(data.x.shape)  #feature
print(data.y.shape)  #label

Data(x=[3700550, 20], edge_attr=[4300999], y=[3700550], train_mask=[857899], valid_mask=[183862], test_mask=[183840], adj_t=[3700550, 3700550, nnz=7994520])
torch.Size([3700550, 20])
torch.Size([3700550])


### 2.4 定义模型
这里我们使用简单的多层感知机作为例子：

In [4]:
class MLP(torch.nn.Module):
    def __init__(self
                 , in_channels
                 , hidden_channels
                 , out_channels
                 , num_layers
                 , dropout
                 , batchnorm=True):
        super(MLP, self).__init__()
        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.batchnorm = batchnorm
        if self.batchnorm:
            self.bns = torch.nn.ModuleList()
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            if self.batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        if self.batchnorm:
            for bn in self.bns:
                bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            if self.batchnorm:
                x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return F.log_softmax(x, dim=-1)


配置后续训练、验证、推理用到的参数。可以调整以下超参以提高模型训练后的验证精度：

- `epochs`：在训练集上训练的代数；
- `lr`：学习率；
- `num_layers`：网络的层数；
- `hidden_channels`：隐藏层维数；
- `dropout`：dropout比例；
- `weight_decay`：正则化项的系数。

In [5]:
mlp_parameters = {
    'lr': 0.01
    , 'num_layers': 2
    , 'hidden_channels': 128
    , 'dropout': 0.0
    , 'batchnorm': False
    , 'weight_decay': 5e-7
                  }
epochs = 200
log_steps =10 # log记录周期


初始化模型，并使用**Area Under the Curve (AUC)** 作为模型评价指标来衡量模型的表现。AUC通过对ROC曲线下各部分的面积求和而得。

具体计算过程参见 https://github.com/scikit-learn/scikit-learn/blob/baf828ca1/sklearn/metrics/_ranking.py#L363

In [6]:
para_dict = mlp_parameters
model_para = mlp_parameters.copy()
model_para.pop('lr')
model_para.pop('weight_decay')
model = MLP(in_channels=data.x.size(-1), out_channels=nlabels, **model_para).to(device)
print(f'Model MLP initialized')


eval_metric = 'auc'  #使用AUC衡量指标
evaluator = Evaluator(eval_metric)


Model MLP initialized


### 2.5 训练

使用训练集中的节点用于训练模型，并使用验证集进行挑选模型。

In [7]:
def train(model, data, train_idx, optimizer):
    # data.y is labels of shape (N, )
    model.train()

    optimizer.zero_grad()
    train_x = data.x[train_idx]
    train_y = data.y[train_idx]
    out = model(train_x)

    loss = F.nll_loss(out, train_y)
    loss.backward()
    optimizer.step()

    return loss.item()


In [8]:
def test(model, data, split_idx, evaluator):
    # data.y is labels of shape (N, )
    with torch.no_grad():
        model.eval()

        losses, eval_results = dict(), dict()
        for key in ['train', 'valid']:
            node_id = split_idx[key]
            out = model(data.x[node_id])
            y_pred = out.exp()  # (N,num_classes)

            losses[key] = F.nll_loss(out, data.y[node_id]).item()
            eval_results[key] = evaluator.eval(data.y[node_id], y_pred)[eval_metric]

    return eval_results, losses, y_pred


In [34]:
print(sum(p.numel() for p in model.parameters()))  #模型总参数量

model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=para_dict['lr'], weight_decay=para_dict['weight_decay'])
best_valid = 0
min_valid_loss = 1e8

data = data.to(device)

for epoch in range(1,epochs + 1):
    loss = train(model, data, train_idx, optimizer)
    eval_results, losses, out = test(model, data, split_idx, evaluator)
    train_eval, valid_eval = eval_results['train'], eval_results['valid']
    train_loss, valid_loss = losses['train'], losses['valid']

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), save_dir+'/model.pt') #将表现最好的模型保存

    if epoch % log_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_eval:.3f}, ' # 我们将AUC值乘上100，使其在0-100的区间内
              f'Valid: {100 * valid_eval:.3f} ')


2946
Epoch: 10, Loss: 0.0822, Train: 64.918, Valid: 65.146 
Epoch: 20, Loss: 0.1017, Train: 67.229, Valid: 67.114 
Epoch: 30, Loss: 0.0814, Train: 69.276, Valid: 68.666 
Epoch: 40, Loss: 0.0690, Train: 70.172, Valid: 69.451 
Epoch: 50, Loss: 0.0651, Train: 69.100, Valid: 68.533 
Epoch: 60, Loss: 0.0654, Train: 69.802, Valid: 69.153 
Epoch: 70, Loss: 0.0646, Train: 70.624, Valid: 69.992 
Epoch: 80, Loss: 0.0645, Train: 70.971, Valid: 70.236 
Epoch: 90, Loss: 0.0643, Train: 70.984, Valid: 70.246 
Epoch: 100, Loss: 0.0642, Train: 71.157, Valid: 70.385 
Epoch: 110, Loss: 0.0641, Train: 71.318, Valid: 70.531 
Epoch: 120, Loss: 0.0641, Train: 71.440, Valid: 70.629 
Epoch: 130, Loss: 0.0640, Train: 71.536, Valid: 70.701 
Epoch: 140, Loss: 0.0640, Train: 71.627, Valid: 70.776 
Epoch: 150, Loss: 0.0639, Train: 71.711, Valid: 70.839 
Epoch: 160, Loss: 0.0639, Train: 71.784, Valid: 70.894 
Epoch: 170, Loss: 0.0639, Train: 71.847, Valid: 70.944 
Epoch: 180, Loss: 0.0639, Train: 71.900, Valid: 70.9

### 2.6 模型预测

In [35]:
model.load_state_dict(torch.load(save_dir+'/model.pt')) #载入验证集上表现最好的模型
def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        out = model(data.x[node_id])
        y_pred = out.exp()  # (N,num_classes)

    return y_pred


In [36]:
dic={0:"正常用户",1:"欺诈用户"}
node_idx = 0
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

node_idx = 1
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')


tensor([0.9957, 0.0043], device='cuda:0')
节点 0 预测对应的标签为:0, 为正常用户。
tensor([0.9743, 0.0257], device='cuda:0')
节点 1 预测对应的标签为:0, 为正常用户。


## 3. 作业评分

**作业要求**：    
                         
1. 请加载你认为训练最佳的模型（不限于图神经网络)
2. 提交的作业包括【程序报告.pdf】和代码文件。

**注意：**
          
1. 在训练模型等过程中如果需要**保存数据、模型**等请写到 **results** 文件夹，如果采用 [离线任务](https://momodel.cn/docs/#/zh-cn/%E5%9C%A8GPU%E6%88%96CPU%E8%B5%84%E6%BA%90%E4%B8%8A%E8%AE%AD%E7%BB%83%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E6%A8%A1%E5%9E%8B) 请务必将模型保存在 **results** 文件夹下。
2. 训练出自己最好的模型后，先按照下列 cell 操作方式实现 NoteBook 加载模型测试；请测试通过在进行【系统测试】。
3. 点击左侧栏`提交作业`后点击`生成文件`则只需勾选 `predict()` 函数的cell，即【**模型预测代码答题区域**】的 cell。
4. 请导入必要的包和第三方库 (包括此文件中曾经导入过的)。
5. 请加载你认为训练最佳的模型，即请按要求填写**模型路径**。
6. `predict()`函数的输入和输出请不要改动。

===========================================  **模型预测代码答题区域**  =========================================== 

在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [ ]:
## 生成 main.py 时请勾选此 cell
from utils import DGraphFin
from utils.evaluator import Evaluator
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.data import Data
import numpy as np
import os

# 这里可以加载你的模型
model = model.load_state_dict(torch.load('./results/model.pt'))

def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """

    # 模型预测时，测试数据已经进行了归一化处理
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        out = model(data.x[node_id])
        y_pred = out.exp()  # (N,num_classes)

    return y_pred


# 4. 其它模型

In [2]:
def load_data(folder, dataset_name, force_to_symmetric: bool = False):
    dataset = DGraphFin(root=folder, name=dataset_name, transform=T.ToSparseTensor())
    # dataset = DGraphFin(root=path, name=dataset_name, transform= T.Compose([T.ToUndirected(),T.ToSparseTensor()]))

    nlabels = dataset.num_classes
    if dataset_name in ['DGraph']:
        nlabels = 2  # 本实验中仅需预测类0和类1

    data = dataset[0]
    if force_to_symmetric:
        data.adj_t = data.adj_t.to_symmetric()  # 将有向图转化为无向图

    if data.edge_index is None:
        row, col, _ = data.adj_t.t().coo()
        data.edge_index = torch.stack([row, col], dim=0)

    if dataset_name in ['DGraph']:
        x = data.x
        x = (x - x.mean(0)) / x.std(0)
        data.x = x
    if data.y.dim() == 2:
        data.y = data.y.squeeze(1)

    split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  # 划分训练集，验证集

    train_idx = split_idx['train']
    return data

In [5]:
from tqdm import tqdm
import os
from pathlib import Path
import pandas as pd
import torch
import torch.nn.functional as F

from models.GraphSAGE import GraphSAGE as Model
# from utils.dgraphfin import load_data
from utils.evaluator import Evaluator


def train(model, data, train_idx, optimizer, cache_path):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.adj_t)

    Path(cache_path).parent.mkdir(parents=True, exist_ok=True)
    torch.save(out, cache_path)

    out = out[train_idx]

    loss = F.nll_loss(out, data.y[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def valid(model, data, split_idx, evaluator, cache_path):
    if os.path.exists(cache_path):
        out = torch.load(cache_path)
    else:
        with torch.no_grad():
            model.eval()
            out = model(data.x, data.adj_t)
    y_pred = out.exp()
    losses, eval_results = dict(), dict()
    for key in ['train', 'valid']:
        node_id = split_idx[key]
        losses[key] = F.nll_loss(out[node_id], data.y[node_id]).item()
        eval_results[key] = evaluator.eval(data.y[node_id], y_pred[node_id])['auc']
    return eval_results, losses


def train_epoch(
    model, data, optimizer, evaluator, lr, min_valid_loss, epoch, model_desc, stop_count,
    use_early_stop=False,
    use_lr_scheduler=False
):
    split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  # 划分训练集，验证集
    cache_path = Path(f'./results/out-{model_desc}.pt')
    cache_path.parent.mkdir(parents=True, exist_ok=True)
    loss = train(model, data, data.train_mask, optimizer, cache_path)
    eval_results, losses = valid(model, data, split_idx, evaluator, cache_path)
    valid_loss = losses['valid']
    early_stop = False
    # 保存最好的模型
    if valid_loss < min_valid_loss:
        stop_count = 0
        model_save_path = Path(f'results/model-{model_desc}.pt')
        model_save_path.parent.mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), model_save_path)
        min_valid_loss = valid_loss
        if cache_path.exists():
            out = torch.load(cache_path)
            torch.save(out, cache_path.with_name(f'out-best-{model_desc}.pt'))
    else:
        stop_count += 1
        if stop_count == 5 and use_lr_scheduler:
            for param_group in optimizer.param_groups:
                lr *= 0.5
                param_group['lr'] = 0.5
        if stop_count == 10 and use_early_stop:
            early_stop = True

    train_log = {
        'epoch': epoch,
        't.loss': losses['train'],
        't.auc': eval_results['train'],
        'v.loss': losses['valid'],
        'v.auc': eval_results['valid'],
        'lr': lr,
        's.cnt': stop_count,
        'min.v.loss': min_valid_loss,
    }
    with open(f'results/train_log-{model_desc}.csv', 'a' if epoch > 0 else 'w', newline='') as f:
        pd.DataFrame({k: [v] for k, v in train_log.items()}).to_csv(f, header=f.tell() == 0, index=False)
    return min_valid_loss, lr, stop_count, early_stop, train_log


if __name__ == '__main__':
    # 设置gpu设备
    device = 0
    device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    print(f'Using device: {device}')

    # 加载数据
    data = load_data('./datasets/632d74d4e2843a53167ee9a1-momodel/', 'DGraph', force_to_symmetric=False)
    data = data.to(device)

    lr = 0.005
    print(f'batch_size: all data, lr: {lr}')

    model_params = {
        "h_c": 32,
        "dropout": 0.0,
        "lr": lr,
        "epochs": 800,
        "layer": 3,
    }

    model = Model(
        in_c=20,
        out_c=2,
        h_c= 32,
        dropout= 0.0,
    )
    model_desc = f'GraphSAGE-{"-".join([f"{k}_{v}" for k, v in model_params.items() ])}'

    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    evaluator = Evaluator('auc')

    min_valid_loss = 1e10
    stop_count = 0

    epoch_iter = tqdm(range(0, 800))
    for epoch in epoch_iter:
        min_valid_loss, lr, stop_count, early_stop, train_log = train_epoch(
            model, data, optimizer, evaluator, lr, min_valid_loss, epoch, model_desc, stop_count
        )
        epoch_iter.set_postfix(**train_log)
    print(f'Training finished with {epoch} epochs')

Using device: cuda:0
batch_size: all data, lr: 0.005


100%|██████████| 800/800 [07:38<00:00,  1.75it/s, epoch=799, lr=0.005, min.v.loss=0.0629, s.cnt=0, t.auc=0.762, t.loss=0.062, v.auc=0.745, v.loss=0.0629] 

Training finished with 799 epochs


In [7]:
from tqdm import tqdm
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import DataLoader

from models.GraphSAGE import GraphSAGE as Model
from utils.evaluator import Evaluator

# 设置gpu设备
device = 0
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

data = load_data('./datasets/632d74d4e2843a53167ee9a1-momodel/', 'DGraph', force_to_symmetric=True)
data = data.to(device)
lr = 0.005
model_params = {
    "h_c": 32,
    "dropout": 0.0,
    "lr": lr,
    "epochs": 800,
    "layer": 3,
}


model = Model(
    in_c=20,
    out_c=2,
    h_c= 32,
    dropout= 0.0,
)
model_desc = f'GraphSAGE-{"-".join([f"{k}_{v}" for k, v in model_params.items() ])}'
model_save_path = f'results/model-{model_desc}.pt'
model.load_state_dict(torch.load(model_save_path, map_location=device))

cache_path = f'./results/out-best-{model_desc}.pt'


def predict(data, node_id):
    if os.path.exists(cache_path):
        out = torch.load(cache_path, map_location=device)
    else:
        with torch.no_grad():
            model.eval()
            out = model(data.x, data.adj_t)

    pred = out[node_id].exp()
    return pred.squeeze(0)


if __name__ == '__main__':
    dic = {0: "正常用户", 1: "欺诈用户"}
    node_idx = 0
    y_pred = predict(data, node_idx)
    print(y_pred)
    print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

    node_idx = 1
    y_pred = predict(data, node_idx)
    print(y_pred)
    print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

tensor([0.9856, 0.0144], device='cuda:0', grad_fn=<SqueezeBackward1>)
节点 0 预测对应的标签为:0, 为正常用户。
tensor([0.9767, 0.0233], device='cuda:0', grad_fn=<SqueezeBackward1>)
节点 1 预测对应的标签为:0, 为正常用户。
